In [1]:
# This notebook is by Anastasia Ruzmaikina for Kaggle Competition Natural Language Processing With Disaster Tweets

Twitter has become an important communication channel in times of emergency. The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

But, it’s not always clear whether a person’s words are actually announcing a disaster.

In this competition, you’re challenged to build a machine learning model that predicts which Tweets are about real disasters and which one’s aren’t. You’ll have access to a dataset of 10,000 tweets that were hand classified.


In this notebook, I use Deberta-V3 from HuggingFace to classify tweets as disaster tweets and non-disaster tweets. This notebook is 81.5% accurate on the competition test dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [10]:
df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df= df.sample(frac = 0.4)
df_train = df.reset_index()
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
df_train

,index,id,keyword,location,text,target
0,2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
1,2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
2,5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
3,132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
4,6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...,...
3040,1873,2692,crush,Everywhere,sevenfigz has a crush: http://t.co/20B3PnQxMD,1
3041,5931,8471,screamed,18 Û¢ CC,Some kids going to leadership camp came into m...,0
3042,2596,3725,destroyed,"ÌÏT: 6.4682,3.18287",Flood: Two people dead 60 houses destroyed in ...,1
3043,1717,2478,collided,NaN,The 2 cars right in front of me collided and I...,1


In [11]:
from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict
df_train['input'] = 'TEXT: '+ df_train.text + ';'
#'TEXT: ' + df_train.text + ';  ANC: '+ df_train.prompt_id# + '; ANC2: '+ df_train.id
#TEXT2: ' + df_train.generated + ';
df_train1 = df_train.drop(['id', 'keyword', 'location'], axis=1)#, 'language'
ds = Dataset.from_pandas(df_train1)
print(ds)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from transformers import DebertaV3Model
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from transformers import TextClassificationPipeline, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments,Trainer
model_nm = '/kaggle/input/debertav3small' # '/kaggle/input/huggingface-deberta-variants/deberta-base-mnli/deberta-base-mnli'  #'/kaggle/input/debertav3small'       #'/kaggle/input/debertav3small'
tokz = AutoTokenizer.from_pretrained(model_nm)
def tok_func(x): return tokz(x["input"])
tok_ds = ds.map(tok_func, batched=True)
tok_ds = tok_ds.rename_columns({'target':'labels'})
dds = tok_ds.train_test_split(0.15, seed=420)
print(dds)
df_test['input'] = 'TEXT: ' + df_test.text  + ';' #'TEXT: ' + df_test.text + '; ANC: ' + df_test.prompt_id#+ '; ANC2: '+ df_test.id
df_test1 = df_test.drop(['id',  'keyword', 'location'], axis=1)
eval_ds = Dataset.from_pandas(df_test1).map(tok_func, batched=True)
bs = 1
epochs = 2
lr = 4.15e-6

Dataset({
    features: ['index', 'text', 'target', 'input'],
    num_rows: 3045
})


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2588
    })
    test: Dataset({
        features: ['index', 'text', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 457
    })
})


Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [12]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=3)

trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz)#, compute_metrics=compute_metrics
trainer.train();
#pipe = TextClassificationPipeline(model=model, tokenizer=tokz)
#prediction = pipe("The text to predict", return_all_scores=True)
#print(prediction)
preds = trainer.predict(eval_ds).predictions.astype(float)
print(preds)
preds = np.clip(preds, 0, 1)

# Make predictions
#predictions = classifier.predict(X_test)

# Evaluate the model (optional)
#classifier.evaluate(X_test)
submission = df_test.id.copy().to_frame()
submission["target"] = np.argmax(preds, axis=1)#classifier.predict(X_test)
#submission = df_test.id.copy().to_frame()
#submission["prediction"] = np.argmax(predictions, axis=1)

#submission["generated"] = submission["generated"].round(1)
#submission.to_csv("/kaggle/working/submission.csv", index=False)
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
print(sample_submission.head())
sample_submission["target"] = np.argmax(preds, axis=1)
sample_submission.to_csv("submission.csv", index=False)
sample_submission.describe()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/debertav3small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,0.948100,1.013741
2,0.801700,1.005341


[[-2.0546875   5.1171875  -3.72851562]
 [-1.76953125  5.07421875 -3.81640625]
 [-2.1015625   5.20703125 -3.6796875 ]
 ...
 [-2.20703125  5.3125     -3.68359375]
 [-2.04296875  5.2265625  -3.76171875]
 [-1.98144531  5.19140625 -3.75976562]]
   id  target
0   0       0
1   2       0
2   3       0
3   9       0
4  11       0


,id,target
count,3263.000000,3263.000000
mean,5427.152927,0.372663
std,3146.427221,0.483588
min,0.000000,0.000000
25%,2683.000000,0.000000
50%,5500.000000,0.000000
75%,8176.000000,1.000000
max,10875.000000,1.000000


In [13]:
sub = pd.read_csv('submission.csv')
sub

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [14]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
#remove_folder_contents(folder_path)
#os.rmdir(folder_path)